# 1、加载库

In [1]:
import common
import imp
imp.reload(common)
from common import *

# 2、获取数据集

In [2]:
%%time
train_dataset = pd.read_csv(TRAIN_DATASET_PATH)
train_label   = pd.read_csv(TRAIN_LABEL_PATH)
test_dataset  = pd.read_csv(TEST_DATASET_PATH)
ip_info       = pd.read_csv(RAW_IP_INFO_PATH)
logger.info([train_dataset.shape, train_label.shape, test_dataset.shape, ip_info.shape])

2022-11-10 11:24:18 <timed exec>[<module>:5] INFO: [(4139070, 7), (140293, 2), (5999605, 7), (1381887, 22)]


Wall time: 27.5 s


# 3、数据预处理

In [3]:
%%time
dataset = pd.concat([train_dataset, test_dataset], axis=0, ignore_index=True)
del train_dataset, test_dataset
gc.collect()

# 去掉列表的左右中括号
dataset['rdata'] = dataset['rdata'].apply(lambda x:x[1:-1])

# 去掉空格
dataset['rdata'] = dataset['rdata'].apply(lambda x:x.replace(' ', ''))

# 时间差
dataset['time_diff'] = dataset['time_last'] - dataset['time_first']

dataset.head()

Wall time: 9.18 s


,count,time_first,time_last,rrname,rrtype,bailiwick,rdata,time_diff
0,222,1592163078,1622858879,0-kgame.com.,A,0-kgame.com.,'59.110.61.64',30695801
1,175,1628061400,1659253644,0-kgame.com.,A,0-kgame.com.,'39.105.158.8',31192244
2,20,1589789678,1591472008,0-kgame.com.,A,0-kgame.com.,'47.244.145.220',1682330
3,1038,1421259590,1622027798,001-design.com.,A,001-design.com.,'114.80.156.15',200768208
4,5,1622540791,1623827206,001-design.com.,A,001-design.com.,'103.21.142.12',1286415


In [29]:
dataset.loc[30, 'rdata']

"'162.159.208.73','162.159.209.73'"

In [28]:
agg_df.loc[30, 'rdata']

list    ['47.103.62.135', '65.19.157.194', '47.103.62....
Name: 30, dtype: object

# 4、特征提取

In [4]:
%%time
agg_df = dataset.groupby(['rrname']).agg({'rdata':[list],
                                            'count':['sum','max', 'min'],
                                            'time_first':['max', 'min'],
                                            'time_last':['max', 'min'],
                                            'rrtype':['unique'],
                                            'bailiwick':[list, 'nunique'],
                                            'time_diff':['sum', 'max', 'min', 'std']}).reset_index()
agg_df.head()

Wall time: 20.9 s


rrname  \
                                  
0                  0-kgame.com.   
1         0000maneras.css2.com.   
2  000differentpieces.css2.com.   
3               001-design.com.   
4              001chuantai.com.   

                                               rdata count            \
                                                list   sum   max min   
0  ['59.110.61.64', '39.105.158.8', '47.244.145.2...   417   222  20   
1                                    ['69.25.142.5']     1     1   1   
2                                    ['69.25.142.5']     2     2   2   
3                 ['114.80.156.15', '103.21.142.12']  1043  1038   5   
4                    ['222.191.251.70', '127.0.0.1']   376   372   4   

   time_first               time_last             rrtype  \
          max         min         max         min unique   
0  1628061400  1589789678  1659253644  1591472008    [A]   
1  1628550051  1628550051  1628550051  1628550051    [A]   
2  1628455998  1628455998  1628455998  1628455998    [A]   
3  1622540791  1421259590  1623827206  1622027798    [A]   
4  1518595697  1442034473  1657696036  1442121329    [A]   

                                    bailiwick          time_diff             \
                                         list nunique        sum        max   
0  [0-kgame.com., 0-kgame.com., 0-kgame.com.]       1   63570375   31192244   
1                                 [css2.com.]       1          0          0   
2                                 [css2.com.]       1          0          0   
3          [001-design.com., 001-design.com.]       1  202054623  200768208   
4        [001chuantai.com., 001chuantai.com.]       1  139187195  139100339   

                          
       min           std  
0  1682330  1.689607e+07  
1        0           NaN  
2        0           NaN  
3  1286415  1.410549e+08  
4    86856  9.829738e+07

In [33]:
agg_df.columns.values

array([('rrname', ''), ('rdata', 'list'), ('count', 'sum'),
       ('count', 'max'), ('count', 'min'), ('time_first', 'max'),
       ('time_first', 'min'), ('time_last', 'max'), ('time_last', 'min'),
       ('rrtype', 'unique'), ('bailiwick', 'list'),
       ('bailiwick', 'nunique'), ('time_diff', 'sum'),
       ('time_diff', 'max'), ('time_diff', 'min'), ('time_diff', 'std')],
      dtype=object)

In [5]:
%%time
agg_df.columns = [''.join(col).strip() for col in agg_df.columns.values]
agg_df.reset_index(drop=True, inplace=True)
agg_df.head()

Wall time: 0 ns


,rrname,rdatalist,countsum,countmax,countmin,time_firstmax,time_firstmin,time_lastmax,time_lastmin,rrtypeunique,bailiwicklist,bailiwicknunique,time_diffsum,time_diffmax,time_diffmin,time_diffstd
0,0-kgame.com.,"['59.110.61.64', '39.105.158.8', '47.244.145.2...",417,222,20,1628061400,1589789678,1659253644,1591472008,[A],"[0-kgame.com., 0-kgame.com., 0-kgame.com.]",1,63570375,31192244,1682330,1.689607e+07
1,0000maneras.css2.com.,['69.25.142.5'],1,1,1,1628550051,1628550051,1628550051,1628550051,[A],[css2.com.],1,0,0,0,NaN
2,000differentpieces.css2.com.,['69.25.142.5'],2,2,2,1628455998,1628455998,1628455998,1628455998,[A],[css2.com.],1,0,0,0,NaN
3,001-design.com.,"['114.80.156.15', '103.21.142.12']",1043,1038,5,1622540791,1421259590,1623827206,1622027798,[A],"[001-design.com., 001-design.com.]",1,202054623,200768208,1286415,1.410549e+08
4,001chuantai.com.,"['222.191.251.70', '127.0.0.1']",376,372,4,1518595697,1442034473,1657696036,1442121329,[A],"[001chuantai.com., 001chuantai.com.]",1,139187195,139100339,86856,9.829738e+07


In [50]:
agg_df.loc[0, 'rdatalist']

["'59.110.61.64'",
 "'39.105.158.8'",
 "'47.244.145.220'",
 "'59.110.61.64'",
 "'39.105.158.8'",
 "'47.244.145.220'"]

In [6]:
%%time
# 去掉两边引号
agg_df['rdatalist'] = agg_df['rdatalist'].apply(lambda x:','.join([i[1:-1] for i in x]).replace('\'','').split(','))

Wall time: 10.7 s


In [52]:
agg_df.loc[0, 'rdatalist']

['59.110.61.64',
 '39.105.158.8',
 '47.244.145.220',
 '59.110.61.64',
 '39.105.158.8',
 '47.244.145.220']

In [7]:
%%time
agg_df['rdatalist_count']     = agg_df['rdatalist'].apply(lambda x:len(x))
agg_df['rdatalist_nunique']   = agg_df['rdatalist'].apply(lambda x:len(set(x)))
agg_df['bailiwicklist_count'] = agg_df['bailiwicklist'].apply(lambda x:len(x))

# 自然数编码
def label_encode(series):
    unique = list(series.unique())
    #unique.sort()
    return series.map(dict(zip(unique, range(series.nunique()))))

for col in ['rrtypeunique']:
    agg_df[col] = label_encode(agg_df[col].astype(str))

Wall time: 7.32 s


# 5、处理ip_info数据集

In [8]:
%%time
ip_info['location'] = ip_info['location'].apply(lambda x: eval(x))
for col in ['continent', 'country', 'province', 'city', 'district', 'lng', 'lat', 'timeZone']:
    ip_info[col] = ip_info['location'].apply(lambda x: x[col])
 
ip_info['asn'] = ip_info['asn'].apply(lambda x: eval(x))
for col in ['number', 'organization', 'operator']:
    ip_info[col] = ip_info['asn'].apply(lambda x: x[col])

ip_info['networkTags'] = ip_info['networkTags'].apply(lambda x:str(x).replace('[',''))
ip_info['networkTags'] = ip_info['networkTags'].apply(lambda x:str(x).replace(']',''))
ip_info['threatTags'] = ip_info['threatTags'].apply(lambda x:str(x).replace('[',''))
ip_info['threatTags'] = ip_info['threatTags'].apply(lambda x:str(x).replace(']',''))

info_cols = ['judgement','networkTags','threatTags','expired','continent','country','province','city',
        'district','timeZone','organization','operator']

for col in info_cols:
    ip_info[col] = ip_info[col].fillna('##')
    ip_info[col] = label_encode(ip_info[col].astype(str))
    ip_info[col] = ip_info[col].astype(str)

ip_info.head()

Wall time: 1min 10s


,ip,judgement,networkTags,threatTags,firstFoundTime,updateTime,expired,openSource,location,asn,...,country,province,city,district,lng,lat,timeZone,number,organization,operator
0,23.55.119.158,0,0,0,2021-01-21T00:00:00,2021-01-21T00:00:00,0,[],"{'continent': '北美洲', 'country': '美国', 'country...","{'number': '16625', 'organization': 'Akamai Te...",...,0,0,0,0,-118.244514,34.051941,0,16625,0,0
1,23.34.103.119,0,0,0,2021-01-21T00:00:00,2021-01-21T00:00:00,0,[],"{'continent': '亚洲', 'country': '日本', 'countryC...","{'number': '16625', 'organization': 'Akamai Te...",...,1,1,1,0,139.808853,35.684078,1,16625,0,0
2,80.239.206.49,0,1,0,NaN,NaN,0,[],"{'continent': '欧洲', 'country': '立陶宛', 'country...","{'number': '1299', 'organization': 'Freewheel ...",...,2,2,2,0,25.269220,54.689830,2,1299,1,1
3,23.196.228.53,0,0,0,2021-01-21T00:00:00,2021-01-21T00:00:00,0,[],"{'continent': '北美洲', 'country': '美国', 'country...","{'number': '35994', 'organization': 'Akamai Te...",...,0,3,3,0,-77.487270,39.043640,3,35994,0,0
4,23.38.35.56,0,0,0,2021-01-21T00:00:00,2021-01-21T00:00:00,0,[],"{'continent': '欧洲', 'country': '荷兰', 'countryC...","{'number': '16625', 'organization': 'Akamai In...",...,3,4,4,0,4.893581,52.379227,4,16625,2,0


# 6、pdns数据集和ip_info数据集结合

In [66]:
ip_info.columns

Index(['ip', 'judgement', 'networkTags', 'threatTags', 'firstFoundTime',
       'updateTime', 'expired', 'openSource', 'location', 'asn', 'samples',
       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'continent',
       'country', 'province', 'city', 'district', 'lng', 'lat', 'timeZone',
       'number', 'organization', 'operator'],
      dtype='object')

In [69]:
col_dict = 'judgement'
"{}_dict = dict(zip(ip_info['ip'], ip_info['{}']))".format(col_dict, col_dict)

"judgement_dict = dict(zip(ip_info['ip'], ip_info['judgement']))"

In [9]:
%%time

col_dicts = ['judgement', 'networkTags', 'threatTags', 'expired', 'continent',
             'country', 'province', 'city', 'district', 'timeZone', 'organization', 'operator']
for col_dict in col_dicts:
    exec("{}_dict = dict(zip(ip_info['ip'], ip_info['{}']))".format(col_dict, col_dict))

Wall time: 7.34 s


In [ ]:
%%time
# 初始化列表
for col_dict in col_dicts:
    exec('{}_li = []'.format(col_dict))

for items in tqdm(agg_df['rdatalist'].values):
    # 置空临时列表
    for col_dict in col_dicts:
        exec('{}_tmp = []'.format(col_dict))
    
    for ip in items:
        try:
            for col_dict in col_dicts:
                exec('{}_tmp.append({}_dict[ip])'.format(col_dict, col_dict))
        except:
            logger.debug('failed, ip {}'.format(ip))
            pass
    
    # 将临时列表添加到主列表
    for col_dict in col_dicts:
        exec('{}_li.append({}_tmp)'.format(col_dict, col_dict))

 26%|██████████████████▋                                                     | 53959/208211 [32:03<32:44:17,  1.31it/s]

In [11]:
%%time
judgement_li = []
networkTags_li = []
threatTags_li = []
expired_li = []
continent_li = []
country_li = []
province_li = []
city_li = []
district_li = []
timeZone_li = []
organization_li = []
operator_li = []
for items in tqdm(agg_df['rdatalist'].values):
    judgement_tmp = []
    networkTags_tmp = []
    threatTags_tmp = []
    expired_tmp = []
    continent_tmp = []
    country_tmp = []
    province_tmp = []
    city_tmp = []
    district_tmp = []
    timeZone_tmp = []
    organization_tmp = []
    operator_tmp = []
    for ip in items:
        try:
            judgement_tmp.append(judgement_dict[ip])
            networkTags_tmp.append(networkTags_dict[ip])
            threatTags_tmp.append(threatTags_dict[ip])
            expired_tmp.append(expired_dict[ip])
            continent_tmp.append(continent_dict[ip])
            country_tmp.append(country_dict[ip])
            province_tmp.append(province_dict[ip])
            city_tmp.append(city_dict[ip])
            district_tmp.append(district_dict[ip])
            timeZone_tmp.append(timeZone_dict[ip])
            organization_tmp.append(organization_dict[ip])
            operator_tmp.append(operator_dict[ip])
        except:
            logger.debug('failed, ip {}'.format(ip))
            pass
    judgement_li.append(judgement_tmp)
    networkTags_li.append(networkTags_tmp)
    threatTags_li.append(threatTags_tmp)
    expired_li.append(expired_tmp)
    continent_li.append(continent_tmp)
    country_li.append(country_tmp)
    province_li.append(province_tmp)
    city_li.append(city_tmp)
    district_li.append(district_tmp)
    timeZone_li.append(timeZone_tmp)
    organization_li.append(organization_tmp)
    operator_li.append(operator_tmp)

100%|█████████████████████████████████████████████████████████████████████████| 208211/208211 [05:47<00:00, 598.44it/s]

Wall time: 5min 59s


# 7、将ip_info提取的特征添加到数据集中

In [12]:
%%time
info_df = pd.DataFrame({'judgement':np.array(judgement_li),'networkTags':np.array(networkTags_li),'threatTags':np.array(threatTags_li),
                        'expired':np.array(expired_li),'continent':np.array(continent_li),'country':np.array(country_li),
                        'province':np.array(province_li),'city':np.array(city_li),'district':np.array(district_li),
                        'timeZone':np.array(timeZone_li),'organization':np.array(organization_li),'operator':np.array(operator_li)})

agg_df = pd.concat([agg_df, info_df], axis=1)
agg_df.head()

Wall time: 1.64 s


,rrname,rdatalist,countsum,countmax,countmin,time_firstmax,time_firstmin,time_lastmax,time_lastmin,rrtypeunique,...,threatTags,expired,continent,country,province,city,district,timeZone,organization,operator
0,0-kgame.com.,"[59.110.61.64, 39.105.158.8, 47.244.145.220]",417,222,20,1628061400,1589789678,1659253644,1591472008,0,...,"[0, 0, 0]","[0, 0, 0]","[1, 1, 1]","[22, 22, 22]","[1337, 1337, 39]","[6246, 6246, 50]","[0, 0, 0]","[12, 12, 12]","[4831, 4831, 4831]","[1918, 1918, 1918]"
1,0000maneras.css2.com.,[69.25.142.5],1,1,1,1628550051,1628550051,1628550051,1628550051,0,...,[0],[0],[0],[0],[19],[22],[0],[0],[9315],[22]
2,000differentpieces.css2.com.,[69.25.142.5],2,2,2,1628455998,1628455998,1628455998,1628455998,0,...,[0],[0],[0],[0],[19],[22],[0],[0],[9315],[22]
3,001-design.com.,"[114.80.156.15, 103.21.142.12]",1043,1038,5,1622540791,1421259590,1623827206,1622027798,0,...,"[0, 25]","[0, 0]","[1, 1]","[22, 22]","[1339, 1339]","[6250, 6250]","[0, 0]","[12, 12]","[185, 5053]","[113, 113]"
4,001chuantai.com.,"[222.191.251.70, 127.0.0.1]",376,372,4,1518595697,1442034473,1657696036,1442121329,0,...,"[0, 0]","[0, 0]","[1, 5]","[22, 28]","[1338, 6]","[6298, 7]","[193, 0]","[12, 15]","[185, 39]","[113, 6]"


In [119]:
dd = agg_df['judgement'].apply(lambda x: ' '.join([str(i) for i in x]))
countv = CountVectorizer(analyzer='char', token_pattern=u"(?u)\b\w+\b")
cv = countv.fit_transform(dd.fillna("##").values)
cv_df = pd.DataFrame(cv.toarray())
cv_df.columns = [col + '_cv_' + str(i) for i in range(cv_df.shape[1])]
cv_df

,operator_cv_0,operator_cv_1,operator_cv_2,operator_cv_3,operator_cv_4
0,5,6,0,0,0
1,3,2,2,0,0
2,3,2,0,2,0
3,13,12,0,2,0
4,9,10,0,0,0
...,...,...,...,...,...
62,5,6,0,0,0
63,3,4,0,0,0
64,5,4,2,0,0
65,1,0,2,0,0


In [123]:
agg_df['judgement']

0                     0 0 0 0 0 0
1                         0 1 0 1
2                         0 2 0 2
3     2 0 0 0 0 0 0 2 0 0 0 0 0 0
4             0 0 0 0 0 0 0 0 0 0
                 ...             
62                    0 0 0 0 0 0
63                        0 0 0 0
64                    1 0 0 1 0 0
65                            1 1
66                    0 0 0 0 0 0
Name: judgement, Length: 67, dtype: object

In [13]:
%%time
small_cols = ['judgement','expired','continent','timeZone']
for col in small_cols:
    print(col)
    agg_df[col] = agg_df[col].apply(lambda x: ' '.join([str(i) for i in x]))

    # CountVectorizer
    countv = CountVectorizer(analyzer='char', token_pattern=u"(?u)\b\w+\b")
    cv = countv.fit_transform(agg_df[col].fillna("##").values)
    cv_df = pd.DataFrame(cv.toarray())
    cv_df.columns = [col + '_cv_' + str(i) for i in range(cv_df.shape[1])]
    cv_df['rrname'] = agg_df['rrname']
    
    agg_df = agg_df.merge(cv_df, on=['rrname'], how='left')

judgement
expired
continent
timeZone
Wall time: 1min 50s


In [14]:
%%time
big_cols = [i for i in info_cols if i not in small_cols]
print(big_cols)
for col in ['rdatalist','bailiwicklist'] + big_cols:
    print(col)
    agg_df[col] = agg_df[col].apply(lambda x: ' '.join([str(i) for i in x]))

    # TfidfVectorizer
    tfidf = TfidfVectorizer()
    tf = tfidf.fit_transform(agg_df[col].fillna("##").values)

    #### TruncatedSVD
    decom = TruncatedSVD(n_components=32, random_state=1024)
    decom_x = decom.fit_transform(tf)
    decom_feas = pd.DataFrame(decom_x)
    decom_feas.columns = [col + '_svd_'+str(i) for i in range(decom_feas.shape[1])]
    decom_feas['rrname'] = agg_df['rrname']

    agg_df = agg_df.merge(decom_feas, on=['rrname'], how='left')

['networkTags', 'threatTags', 'country', 'province', 'city', 'district', 'organization', 'operator']
rdatalist
bailiwicklist
networkTags
threatTags
country
province
city
district
organization
operator
Wall time: 6min 19s


In [84]:
agg_df.columns

Index(['rrname', 'rdatalist', 'countsum', 'countmax', 'countmin',
       'time_firstmax', 'time_firstmin', 'time_lastmax', 'time_lastmin',
       'rrtypeunique',
       ...
       'bailiwicklist_svd_22', 'bailiwicklist_svd_23', 'bailiwicklist_svd_24',
       'bailiwicklist_svd_25', 'bailiwicklist_svd_26', 'bailiwicklist_svd_27',
       'bailiwicklist_svd_28', 'bailiwicklist_svd_29', 'bailiwicklist_svd_30',
       'bailiwicklist_svd_31'],
      dtype='object', length=115)

# 8、训练数据/测试数据准备

In [15]:
%%time
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

Wall time: 0 ns


In [16]:
%%time
train_label.columns = ['rrname','label']
agg_df = agg_df.merge(train_label, on=['rrname'], how='left')
agg_df = reduce_mem_usage(agg_df)

features = [f for f in agg_df.columns if f not in ['rrname','label','rdatalist','bailiwicklist']+info_cols]

train = agg_df[~agg_df.label.isnull()].reset_index(drop=True)
test = agg_df[agg_df.label.isnull()].reset_index(drop=True)

x_train = train[features]
x_test = test[features]

y_train = train['label']

del agg_df
gc.collect()

print(x_train.shape, x_test.shape)

Mem. usage decreased to 183.47 Mb (69.8% reduction)
(140293, 366) (67918, 366)
Wall time: 2min 50s


# 9、模型训练
- 直接构建了一个函数，可以调用三种树模型，方便快捷

In [17]:
%%time
def cv_model(clf, train_x, train_y, test_x, clf_name):
    folds = 5
    seed = 2022
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

    oof = np.zeros(train_x.shape[0])
    pred = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.01,
                'seed': 2022,
                'n_jobs':-1,
                'verbose': -1,
            }

            model = clf.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], 
                              categorical_feature=[], verbose_eval=500, early_stopping_rounds=500)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
            
            # print(list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
                
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            test_matrix = clf.DMatrix(test_x)
            
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 5,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.01,
                      'tree_method': 'exact',
                      'seed': 2022,
                      'nthread': 36
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(params, train_matrix, num_boost_round=50000, evals=watchlist, verbose_eval=500, early_stopping_rounds=200)
            val_pred  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            test_pred = model.predict(test_matrix , ntree_limit=model.best_ntree_limit)
            
        if clf_name == "cat":
            params = {'learning_rate': 0.01, 'depth': 5, 'l2_leaf_reg': 10, 'bootstrap_type': 'Bernoulli',
                      'od_type': 'Iter', 'od_wait': 50, 'random_seed': 11, 'allow_writing_files': False}
            
            model = clf(iterations=20000, **params)
            model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                      cat_features=[], use_best_model=True, verbose=500)
            
            val_pred  = model.predict(val_x)
            test_pred = model.predict(test_x)
            
        oof[valid_index] = val_pred
        pred += test_pred / kf.n_splits
        cv_scores.append(f1_score(val_y, pd.DataFrame(val_pred)[0].apply(lambda x: 1 if x>0.35 else 0), average='macro'))
        
        print(cv_scores)
        
    print("%s_train_score_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return oof, pred

Wall time: 0 ns


In [18]:
def lgb_model(x_train, y_train, x_test):
    return cv_model(lgb, x_train, y_train, x_test, "lgb")

def xgb_model(x_train, y_train, x_test):
    return  cv_model(xgb, x_train, y_train, x_test, "xgb")

def cat_model(x_train, y_train, x_test):
    return cv_model(CatBoostRegressor, x_train, y_train, x_test, "cat") 

In [19]:
%%time
cat_train, cat_test = cat_model(x_train, y_train, x_test)

************************************ 1 ************************************
0:	learn: 0.1216046	test: 0.1216328	best: 0.1216328 (0)	total: 2.04s	remaining: 11h 19m 59s
500:	learn: 0.0045110	test: 0.0045206	best: 0.0045206 (500)	total: 28.5s	remaining: 18m 27s
1000:	learn: 0.0038936	test: 0.0043145	best: 0.0043145 (996)	total: 45.8s	remaining: 14m 30s
1500:	learn: 0.0034790	test: 0.0042692	best: 0.0042692 (1500)	total: 1m 9s	remaining: 14m 21s
2000:	learn: 0.0031379	test: 0.0042355	best: 0.0042355 (2000)	total: 1m 28s	remaining: 13m 12s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.004229003516
bestIteration = 2109

Shrink model to first 2110 iterations.
[1.0]
************************************ 2 ************************************
0:	learn: 0.1215993	test: 0.1216322	best: 0.1216322 (0)	total: 41.6ms	remaining: 13m 52s
500:	learn: 0.0033396	test: 0.0084146	best: 0.0084146 (500)	total: 18.2s	remaining: 11m 50s
Stopped by overfitting detector  (50 iterations wait)

In [20]:
test['label'] = cat_test
test['label'] = test['label'].apply(lambda x: 1 if x>0.35 else 0)
# 提交最终结果
test[['rrname','label']].to_csv("result.csv", index=False, header=False)

In [95]:
train[features].describe()

,countsum,countmax,countmin,time_firstmax,time_firstmin,time_lastmax,time_lastmin,rrtypeunique,bailiwicknunique,time_diffsum,...,bailiwicklist_svd_22,bailiwicklist_svd_23,bailiwicklist_svd_24,bailiwicklist_svd_25,bailiwicklist_svd_26,bailiwicklist_svd_27,bailiwicklist_svd_28,bailiwicklist_svd_29,bailiwicklist_svd_30,bailiwicklist_svd_31
count,67.000000,67.000000,67.000000,6.700000e+01,6.700000e+01,6.700000e+01,6.700000e+01,67.0,67.0,6.700000e+01,...,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,6.700000e+01,67.000000,67.000000
mean,1994.597015,785.791045,154.746269,1.574655e+09,1.447029e+09,1.649654e+09,1.513904e+09,0.0,1.0,3.504921e+08,...,0.000003,-0.000001,-0.000003,-0.000004,0.000005,-0.000005,0.000003,-4.172325e-07,-0.000002,0.000003
std,1098.884780,488.002780,314.518263,7.374909e+07,8.839525e+07,3.035030e+07,1.150502e+08,0.0,0.0,1.672771e+08,...,0.119324,0.119385,0.119324,0.119324,0.119324,0.119324,0.119324,1.193848e-01,0.119385,0.119324
min,20.000000,5.000000,1.000000,1.353203e+09,1.280369e+09,1.511016e+09,1.280627e+09,0.0,1.0,1.026444e+06,...,-0.307861,-0.346924,-0.270508,-0.319580,-0.260986,-0.283447,-0.340088,-2.543945e-01,-0.266846,-0.274902
25%,1173.000000,430.500000,3.000000,1.537303e+09,1.386099e+09,1.659254e+09,1.432600e+09,0.0,1.0,2.436395e+08,...,-0.033707,-0.023270,-0.068817,-0.061615,-0.063522,-0.056686,-0.051544,-4.144287e-02,-0.070068,-0.049606
50%,2086.000000,736.000000,13.000000,1.589480e+09,1.459108e+09,1.659289e+09,1.513584e+09,0.0,1.0,3.608983e+08,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
75%,2700.000000,1002.500000,137.000000,1.634497e+09,1.508536e+09,1.659294e+09,1.622702e+09,0.0,1.0,4.534946e+08,...,0.055176,0.052719,0.062653,0.048615,0.046387,0.037567,0.052368,3.625488e-02,0.078979,0.055756
max,4906.000000,2447.000000,1520.000000,1.657582e+09,1.655721e+09,1.659295e+09,1.659295e+09,0.0,1.0,7.482811e+08,...,0.309082,0.382324,0.281250,0.335938,0.347168,0.365723,0.369629,3.132324e-01,0.324951,0.340088


In [96]:
test[features].describe()

,countsum,countmax,countmin,time_firstmax,time_firstmin,time_lastmax,time_lastmin,rrtypeunique,bailiwicknunique,time_diffsum,...,bailiwicklist_svd_22,bailiwicklist_svd_23,bailiwicklist_svd_24,bailiwicklist_svd_25,bailiwicklist_svd_26,bailiwicklist_svd_27,bailiwicklist_svd_28,bailiwicklist_svd_29,bailiwicklist_svd_30,bailiwicklist_svd_31
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
